## 协程

相较于Python的其他特性，协程的资料较少（在阅读这一章之前，本人之前从未有意识的使用过）。从句法上来看协程与生成器类似，这两者均是包含yield关键字的“函数”。不同的是，协程中yield通常出现在表达式的右侧 —— 可以产出值也可以不产出值。协程可能会从调用方接收数据，这一过程由调用方主动使用.send()方法提供

从功能上来看，协程中的yield关键字是一种流程控制工具，以实现协作式多任务。

## 协程的基本行为

Python中协程的正常运作依赖于生成器
1. 协程使用生成器函数定义，即定义体中包含yield关键字
2. 对于协程，yield关键字在表达式中使用，并且能够从调用方接收数据，默认值为None
3. 协程的运作方式类似于生成器 —— 首先需要得到生成器对象，并且使用next()函数调用该对象后会运行到第一个yield语句
4. 当程序运行到定义体末尾后，生成器同样会抛出StopIteration

In [3]:
def simple_coroutine():
    print("-> coroutine started")
    x = yield
    print("-> coroutine received: ", x)

my_coro = simple_coroutine()
print(type(my_coro))
next(my_coro)
my_coro.send(1024)

<class 'generator'>
GEN_CREATED
-> coroutine started
GEN_SUSPENDED
-> coroutine received:  1024


StopIteration: 

上述例子表明协程可能处于多种状态，这些状态可以由inspect.getgeneratorstate()函数获得：
* GEN_CREATED：等待开始执行
* GEN_RUNNING：解释器正在执行
* GEN_SUSPENDED：在yield表达式处暂停
* GEN_CLOSED：执行结束

值得注意的是，仅当生成器处于GEN_SUSPENDED状态时才可以使用.send()传递数据。若在GEN_CREATED状态时传递则会抛出TypeError: can't send non-None value to a just-started generator；若在GEN_CLOSED阶段传递则会抛出StopIteration。

### 协程的执行过程

协程的执行过程需要着重理解。关键在于暂停的位置以及.send()传递数据的时机。

实际上也就一点：对于生成器，若调用next()函数，则**该生成器会运行到下一个yield语句处（包含yield语句），.send()语句传递数据在此之后**

下述例子描述了协程的基本执行过程，本书的图16-1对协程的执行过程有很清晰的描述

In [7]:
def sample_coro2(a):
    print("-> started: a = : ", a)
    b = yield a
    print("-> Received: b = : ", b)
    c = yield a + b
    print("-> Received: c = : ", c)

my_coro2 = sample_coro2(14)
print("开始执行")
print(next(my_coro2))
print("第一次send：")
print(my_coro2.send(28))
print("第二次send：")
print(my_coro2.send(99))

开始执行
-> started: a = :  14
14
第一次send：
-> Received: b = :  28
42
第二次send：
-> Received: c = :  99


StopIteration: 

## 预激协程

协程使用前需要进行预激，即让生成器处于GEN_SUSPENDED状态。

预激协程最为朴素的方法就是通过next()函数，使得生成器运行到第一个yield位置。为了简化起见，还可以通过装饰器预激协程。具体来说，通过实现一个装饰器自动运行next()函数。

用于预激协程的装饰器需要自行实现，本书给出了一个例子

In [8]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        # 创建生成器
        gen = func(*args, **kwargs)
        # 预激
        next(gen)
        # 返回生成器
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg = averager()
print(coro_avg.send(10))
print(coro_avg.send(30))
print(coro_avg.send(50))


10.0
20.0
30.0


## 终止协程和异常处理

协程中没有处理的异常会向上冒泡至调用方，并且终止该协程。

上述过程实际上暗示了某种终止协程的方法：通过传递某些特殊的数据，让协程报错后自动终止。例如传递None、StopIteration等

更“温和”的做法是通过两个方法显式的传递异常给协程：
1. throw
.throw(exc_type[, exc_value[, traceback]])，显式传递错误给协程，若协程处理了该错误则会自动运行到下一个yield位置并返回产出的值；否则该异常会向上冒泡至调用方
2. close
强制协程在暂停处抛出GeneratorExit，若协程没有处理该错误则协程终止并且调用方不会报错